# JMESPath
---

JMESPath (pronounced "james path") allows you to declaratively specify how to extract elements from a JSON document.

## Installation

You can install `jmespath` using `pip`, like this:

In [1]:
!pip install jmespath

You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


## Usage

The jmespath.py library has two functions that operate on python data structures. You can use search and give it the jmespath expression and the data:

In [8]:
import jmespath

In [9]:
jmespath.search('foo.bar', {'foo': {'bar': 'baz'}})

'baz'

Similar to the re module, you can use the compile function to compile the JMESPath expression and use this parsed expression to perform repeated searches:

In [10]:
expression = jmespath.compile('foo.bar')
expression.search({'foo': {'bar': 'baz'}})

'baz'

In [11]:
expression.search({'foo': {'bar': 'other'}})

'other'

This is useful if you're going to use the same jmespath expression to search multiple documents. This avoids having to reparse the JMESPath expression each time you search a new document.

### Options
You can provide an instance of jmespath.Options to control how a JMESPath expression is evaluated. The most common scenario for using an Options instance is if you want to have ordered output of your dict keys. To do this you can use either of these options:

In [16]:
import collections

mydata = {'foo': {'bar': 'baz'}}
jmespath.search(
    '{a: a, b: b}',
    mydata,
    jmespath.Options(dict_cls=collections.OrderedDict)
)

OrderedDict([('a', None), ('b', None)])

In [17]:
parsed = jmespath.compile('{a: a, b: b}')
parsed.search(
    mydata,
    jmespath.Options(dict_cls=collections.OrderedDict)
)

OrderedDict([('a', None), ('b', None)])

### Custom Functions

The JMESPath language has numerous built-in functions, but it is also possible to add your own custom functions. Keep in mind that custom function support in jmespath.py is experimental and the API may change based on feedback.

If you have a custom function that you've found useful, consider submitting it to jmespath.site and propose that it be added to the JMESPath language. You can submit proposals here.

To create custom functions:

Create a subclass of jmespath.functions.Functions.
Create a method with the name _func_<your function name>.
Apply the jmespath.functions.signature decorator that indicates the expected types of the function arguments.
Provide an instance of your subclass in a jmespath.Options object.
Below are a few examples:

In [18]:
from jmespath import functions

# 1. Create a subclass of functions.Functions.
#    The function.Functions base class has logic
#    that introspects all of its methods and automatically
#    registers your custom functions in its function table.
class CustomFunctions(functions.Functions):

    # 2 and 3.  Create a function that starts with _func_
    # and decorate it with @signature which indicates its
    # expected types.
    # In this example, we're creating a jmespath function
    # called "unique_letters" that accepts a single argument
    # with an expected type "string".
    @functions.signature({'types': ['string']})
    def _func_unique_letters(self, s):
        # Given a string s, return a sorted
        # string of unique letters: 'ccbbadd' ->  'abcd'
        return ''.join(sorted(set(s)))

    # Here's another example.  This is creating
    # a jmespath function called "my_add" that expects
    # two arguments, both of which should be of type number.
    @functions.signature({'types': ['number']}, {'types': ['number']})
    def _func_my_add(self, x, y):
        return x + y

In [19]:
# 4. Provide an instance of your subclass in a Options object.
options = jmespath.Options(custom_functions=CustomFunctions())

In [20]:
# Provide this value to jmespath.search:
# This will print 3
print(
    jmespath.search(
        'my_add(`1`, `2`)', {}, options=options)
)

3


In [21]:
# This will print "abcd"
print(
    jmespath.search(
        'foo.bar | unique_letters(@)',
        {'foo': {'bar': 'ccbbadd'}},
        options=options)
)

abcd


Again, if you come up with useful functions that you think make sense in the JMESPath language (and make sense to implement in all JMESPath libraries, not just python), please let us know at jmespath.site.